In [3]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
    
# load in data and labels    
data = np.array(np.loadtxt('data/data.txt'))